In [1]:
import warnings

from docutils.nodes import classifier

warnings.filterwarnings("ignore")

from fss_funcs import fss
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
import numpy as np
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
from sklearn.tree import DecisionTreeClassifier
import wittgenstein as lw
from sklearn.model_selection import train_test_split
import time
from bayes_optimizable_mlpc import BayesOptimizableMLPC
from io import StringIO

rng = np.random.RandomState(42)

In [2]:
df = pd.read_csv("csv/outlier_filtered.csv")

response_var = 'Diabetes_012'
features = list(df.columns)
features.remove(response_var)

print(features, response_var)
# Pretty-print using tabulate
df.head(1)

['BMI_q_normal', 'MentHlth', 'PhysHlth_q_uniform', 'GenHlth_q_uniform', 'Age_q_uniform', 'Education_coxbox', 'Income_q_uniform', 'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex'] Diabetes_012


,BMI_q_normal,MentHlth,PhysHlth_q_uniform,GenHlth_q_uniform,Age_q_uniform,Education_coxbox,Income_q_uniform,Diabetes_012,HighBP,HighChol,...,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,DiffWalk,Sex
0,1.60221,1.998592,0.891892,1.0,0.581582,-1.109347,0.117117,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


Features kept in multivariate feature selection
- K=08 -> [0, 2, 3, 4, 6, 7, 8, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 19]

Features kept in univariate feature selection
- K=08 -> [0, 3, 4, 6, 7, 8, 12, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 19]


In [31]:
"""
Features kept in multivariate feature selection
- K=08 -> [0, 2, 3, 4, 6, 7, 8, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 19]

Features kept in univariate feature selection
- K=08 -> [0, 3, 4, 6, 7, 8, 12, 19]
- K=10 -> [0, 2, 3, 4, 5, 6, 7, 8, 12, 19]
- K=12 -> [0, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 19]
- K=14 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19]
- K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 19]

Wrapper
K=08 -> [0, 3, 4, 6, 7, 8, 12, 16]
K=10 -> [0, 3, 4, 6, 7, 8, 9, 12, 16, 20]
K=12 -> [0, 3, 4, 6, 7, 8, 9, 11, 12, 16, 17, 19]
K=14 -> [0, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 19]
K=16 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 19]
"""
fea_subsets = {
    'multivariate':{8:[0, 2, 3, 4, 6, 7, 8, 19],
               10:[0, 2, 3, 4, 5, 6, 7, 8, 12, 19],
               12:[0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 19],
               14:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19],
               16:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 19]},
    'univariate':{8:[0, 3, 4, 6, 7, 8, 12, 19],
                  10:[0, 2, 3, 4, 5, 6, 7, 8, 12, 19],
                  12:[0, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 19],
                  14:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19],
                  16:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 19]},
    'wrapper':{8:[0, 3, 4, 6, 7, 8, 12, 16],
               10:[0, 3, 4, 6, 7, 8, 9, 12, 16, 20],
               12:[0, 3, 4, 6, 7, 8, 9, 11, 12, 16, 17, 19],
               14:[0, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 19],
               16:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 1]},
    'original':{21: np.arange(len(features))},
}


models = {
    'K-nearest neighbors' : BayesSearchCV(KNeighborsClassifier(),{
            'n_neighbors': Integer(3, 20),
            'weights': Categorical(['uniform', 'distance']),
            'metric': Categorical(['euclidean', 'manhattan', 'minkowski'])},
            n_iter=10,
            random_state=rng,
            n_jobs=-1,
            n_points=2,
            scoring='f1_macro',
        ),
    'Artificial neural networks': BayesSearchCV(
            BayesOptimizableMLPC(max_iter=1000,warm_start=True, random_state=rng, solver='adam'),
            search_spaces={
                'hidden_layer_sizes': Categorical(["32", "64", "32, 32", "64, 32", "64, 64", "32, 64, 32"]),  # Different layer structures
                'activation': Categorical(['identity', 'logistic', 'tanh', 'relu']),
                'alpha': Real(1e-4, 1e-2, prior='log-uniform'),  # L2 regularization term
                'batch_size': Integer(256, 2056),  # Size of minibatches for stochastic optimizers
            },
            random_state=rng,
            scoring='f1_macro',
            n_jobs=-1,
            n_points=4, 
            n_iter=10
        ),
'Support vector machines':  BayesSearchCV(
            SVC(random_state=rng,cache_size=1000),
            {
                'C': Real(1e-2, 1e2, prior='uniform'),
                'kernel': Categorical(['linear', 'rbf']),
                'gamma': Categorical(['scale', 'auto'])
            },
            random_state=rng,
            n_jobs=-1,
            n_points=4,
            scoring='f1_macro',
            cv=2,
            n_iter=2,
        ),
'Classification trees': BayesSearchCV(
            DecisionTreeClassifier(class_weight='balanced', random_state=rng),
            {
                'criterion': Categorical(['gini', 'entropy', 'log_loss']),
                'splitter': Categorical(['best', 'random']),
                'max_depth': Integer(5, 50),
                'min_samples_split': Real(1e-6, 0.1),
                'min_samples_leaf': Real(1e-6, 0.1),
                'class_weight': Categorical(['balanced']),
                'ccp_alpha': Real(0, 1),
            },
            random_state=rng,
            scoring='f1_macro',
            n_jobs=-1,
            n_points=4,
            n_iter=100
       )
}

binary_models = {
    "Rule Induction": BayesSearchCV(
        lw.RIPPER(random_state=0),
        search_spaces={
            'k': Integer(1, 5),                           # Reduce optimization rounds
            'prune_size': Real(0.1, 0.3),                 # Narrow pruning range
            'dl_allowance': Integer(16, 64),              # Smaller early stopping range
            'n_discretize_bins': Integer(5, 15),          # Fewer discretization bins
            'max_rules': Integer(1, 10),                  # Lower number of rules
            'min_rule_samples': Integer(1, 5),            # Smaller minimum samples for rule growth
            'min_ruleset_samples': Integer(1, 10),        # Lower min samples for ruleset
            'max_rule_conds': Integer(1, 6),              # Fewer conditions per rule
            'max_total_conds': Integer(10, 30),           # Lower total conditions
        },
        random_state=rng,
        scoring='f1_macro',
        n_jobs=-1,
        n_points=4,
        n_iter=4,  # Limit number of iterations
    ),
}

# RIPPER PARAMS
"""
k : int, default=2
    Number of RIPPERk optimization iterations.
prune_size : float, default=.33
    Proportion of training set to be used for pruning.
dl_allowance : int, default=64
    Terminate Ruleset grow phase early if a Ruleset description length is encountered
    that is more than this amount above the lowest description length so far encountered.
n_discretize_bins : int, default=10
    Fit apparent numeric attributes into a maximum of n_discretize_bins discrete bins, inclusive on upper part of range. Pass None to disable auto-discretization.
random_state : int, default=None
    Random seed for repeatable results.

Limits for early-stopping. Intended for enhancing model interpretability and limiting training time on noisy datasets. Not specifically intended for use as a hyperparameter, since pruning already occurs during training, though it is certainly possible that tuning could improve model performance.
max_rules : int, default=None
    Maximum number of rules.
min_rule_samples : int, defulat=None
    Minimum number of covered samples per rule before halting rule growth.
min_ruleset_samples : int, default=None
    Minimum number of samples per rule before halting ruleset growth.
max_rule_conds : int, default=None
    Maximum number of conds per rule.
max_total_conds : int, default=None
    Maximum number of total conds in entire ruleset.
"""
pass

In [36]:
['AnyHealthcare'] # Wrapper
['HvyAlcoholConsump'] # Multivariate
['Veggies'] #Univariate

['NoDocbcCost', 'Fruits', 'Sex'] # None chose
np.array(features)[[18,14,20]]

array(['NoDocbcCost', 'Fruits', 'Sex'], dtype='<U20')

In [29]:
X_total, y_total = df[features], df[response_var]

# Define your desired fraction
fraction = 1

if fraction == 1:
    X_fraction, y_fraction = X_total, y_total
else:
    # Stratified sampling using train_test_split with only one split (fraction is the test_size here)
    X_fraction, _, y_fraction, _ = train_test_split(
        X_total, y_total,
        test_size=1-fraction,
        random_state=rng,
        stratify=y_total
    )

X_train, X_test, y_train, y_test = train_test_split(X_fraction, y_fraction,
                                                    train_size=0.9,
                                                    random_state=rng,
                                                    stratify=y_fraction)
print(X_train.shape, y_train.shape)

(228281, 21) (228281,)


In [30]:
# Start tracking total runtime
def run_models(elapse=0, skip=[]):
    start_total = time.time()
    
    y = y_train
    it = 0
    
    for model in models.keys():
        if model in skip:
            print(f"Skipping {model}")
            continue 
            
        for fss_strategy in fea_subsets.keys():
            if fss_strategy not in ['original']:
                continue 
                
            for k in fea_subsets[fss_strategy].keys():
                if it < elapse:
                    print(f"Skipping it={it} with {model};{fss_strategy};{k};")
                    it += 1
                    continue 
                    
                # Start tracking time for this iteration
                start_time = time.time()
                
                # Define the feature subset and input data
                fea_subset = np.array(features)[fea_subsets[fss_strategy][k]]
                X = X_train[fea_subset]
                
                # Fit the model
                bayes_search = models[model]
                print(f"\rFitting {model}...                   ", end="")
                if model == "Support vector machines":
                    X_sub, _, y_sub, _ = train_test_split(X,y, train_size=0.05, random_state=rng, stratify=y)
                    bayes_search.fit(X_sub, y_sub)
                else:
                    bayes_search.fit(X, y)
                print(f"\rFitting {model} completed            ", end="")
                # Get best parameters and score
                best_params = bayes_search.best_params_
                score = bayes_search.score(X_test[fea_subset], y_test)
                
                # Calculate elapsed time for this iteration
                elapsed_time = time.time() - start_time
                
                # Print results along with time taken
                print(f"\r{it};{model};{fss_strategy};{k};{best_params};{score:01.04f};{elapsed_time:06.02f}")
                it += 1
    
    # Calculate and print total runtime
    total_time = time.time() - start_total
    print(f"Total runtime: {total_time:.2f} seconds")

run_models(0)

0;K-nearest neighbors;original;21;OrderedDict({'metric': 'manhattan', 'n_neighbors': 4, 'weights': 'distance'});0.3977;669.45
1;Artificial neural networks;original;21;OrderedDict({'activation': 'relu', 'alpha': 0.004639602632329191, 'batch_size': 457, 'hidden_layer_sizes': '64, 32'});0.3995;680.02
2;Support vector machines;original;21;OrderedDict({'C': 71.79275504420298, 'gamma': 'scale', 'kernel': 'rbf'});0.4037;022.34
3;Classification trees;original;21;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 23, 'min_samples_leaf': 0.05741150768628191, 'min_samples_split': 0.1, 'splitter': 'best'});0.4138;141.39
Total runtime: 1513.20 seconds


In [20]:
from sklearn.model_selection import cross_val_score

# Start tracking total runtime

y_ttrain = y_train.copy()
y_ttrain[y_ttrain == 2] = 1
y_ttest = y_test.copy()
y_ttest[y_ttest == 2] = 1



def run_binary_models(elapse=0):
    start_total = time.time()
    
    y = y_ttrain
    it = 0
    
    for model in binary_models.keys():
        for fss_strategy in fea_subsets.keys():
            for k in fea_subsets[fss_strategy].keys():
                if it < elapse:
                    print(f"Skipping it={it} with {model};{fss_strategy};{k};")
                    it += 1
                    continue 
                    
                # Start tracking time for this iteration
                start_time = time.time()
                
                # Define the feature subset and input data
                fea_subset = np.array(features)[fea_subsets[fss_strategy][k]]
                X = X_train[fea_subset]
                
                
                # Fit the model
                bayes_search = binary_models[model]
                print(f"\rFitting {model}...                   ", end="")
                bayes_search.fit(X, y)
                print(f"\rFitting {model} completed            ", end="")
                # Get best parameters and score
                best_params = bayes_search.best_params_
                score = bayes_search.score(X_test[fea_subset], y_ttest)
                
                # Calculate elapsed time for this iteration
                elapsed_time = time.time() - start_time
                
                # Print results along with time taken
                print(f"\r{it};{model};{fss_strategy};{k};{best_params};{score:01.04f};{elapsed_time:06.02f}")
                it += 1
    
    # Calculate and print total runtime
    total_time = time.time() - start_total
    print(f"Total runtime: {total_time:.2f} seconds")


# Start the timer
start_time = time.time()
ripper = lw.RIPPER(random_state=0)
cv_scores = cross_val_score(ripper, X_train, y_ttrain, cv=5)
elapsed_time = time.time() - start_time
print(f"Estimated training time for 5-fold CV without optimizer: {elapsed_time:.2f} seconds")




run_binary_models(0)

KeyboardInterrupt: 

In [6]:
output = """
it;model;fss strategy;num fea;best parameters;score;elapsed time 
0;K-nearest neighbors;wrapper;8;OrderedDict({'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'});0.4053;197.50
1;K-nearest neighbors;wrapper;10;OrderedDict({'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'});0.3999;276.22
2;K-nearest neighbors;wrapper;12;OrderedDict({'metric': 'minkowski', 'n_neighbors': 3, 'weights': 'distance'});0.4063;314.40
3;K-nearest neighbors;wrapper;14;OrderedDict({'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'});0.4044;498.63
4;K-nearest neighbors;wrapper;16;OrderedDict({'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'});0.4109;1793.14
10;Classification trees;multivariate;8;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.0551451310059547, 'min_samples_split': 0.1, 'splitter': 'random'});0.4217;049.93
11;Classification trees;multivariate;10;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.05781319163758467, 'min_samples_split': 0.08305687894819609, 'splitter': 'best'});0.4138;065.82
12;Classification trees;multivariate;12;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1e-06, 'min_samples_split': 0.045587443653603926, 'splitter': 'random'});0.4191;043.55
13;Classification trees;multivariate;14;OrderedDict({'ccp_alpha': 0.010679273044110017, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 8, 'min_samples_leaf': 0.1, 'min_samples_split': 0.006038931162415052, 'splitter': 'random'});0.4021;054.14
14;Classification trees;multivariate;16;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 50, 'min_samples_leaf': 1e-06, 'min_samples_split': 0.1, 'splitter': 'best'});0.4069;052.86
15;Classification trees;univariate;8;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 47, 'min_samples_leaf': 0.05915872872692148, 'min_samples_split': 0.1, 'splitter': 'random'});0.4081;045.71
16;Classification trees;univariate;10;OrderedDict({'ccp_alpha': 0.0008041567470032194, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 44, 'min_samples_leaf': 0.014046579680346154, 'min_samples_split': 0.028138888173554914, 'splitter': 'random'});0.4151;042.24
17;Classification trees;univariate;12;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 0.05464509991436246, 'min_samples_split': 1e-06, 'splitter': 'best'});0.4175;042.57
18;Classification trees;univariate;14;OrderedDict({'ccp_alpha': 0.0004095218216206843, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 13, 'min_samples_leaf': 0.03518870088484663, 'min_samples_split': 0.012278844580237428, 'splitter': 'random'});0.4141;053.75
19;Classification trees;univariate;16;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 5, 'min_samples_leaf': 1e-06, 'min_samples_split': 1e-06, 'splitter': 'random'});0.4230;070.15
0;Artificial neural networks;multivariate;8;OrderedDict({'activation': 'relu', 'alpha': 0.00034140471672701174, 'batch_size': 1666, 'hidden_layer_sizes': '64, 64'});0.4177;062.86
1;Artificial neural networks;multivariate;10;OrderedDict({'activation': 'relu', 'alpha': 0.003419694880629917, 'batch_size': 1669, 'hidden_layer_sizes': '64, 64'});0.4267;089.01
2;Artificial neural networks;multivariate;12;OrderedDict({'activation': 'identity', 'alpha': 0.008980741502394047, 'batch_size': 530, 'hidden_layer_sizes': '32, 32'});0.4093;033.12
3;Artificial neural networks;multivariate;14;OrderedDict({'activation': 'tanh', 'alpha': 0.00036249930490987373, 'batch_size': 1405, 'hidden_layer_sizes': '64, 32'});0.4504;124.56
4;Artificial neural networks;multivariate;16;OrderedDict({'activation': 'relu', 'alpha': 0.0002043075311149019, 'batch_size': 733, 'hidden_layer_sizes': '64'});0.4255;071.89
5;Artificial neural networks;univariate;8;OrderedDict({'activation': 'identity', 'alpha': 0.009090263265822638, 'batch_size': 256, 'hidden_layer_sizes': '64'});0.4168;039.34
6;Artificial neural networks;univariate;10;OrderedDict({'activation': 'identity', 'alpha': 0.0001, 'batch_size': 290, 'hidden_layer_sizes': '32, 32'});0.4031;065.41
7;Artificial neural networks;univariate;12;OrderedDict({'activation': 'relu', 'alpha': 0.0022471872460717193, 'batch_size': 1730, 'hidden_layer_sizes': '32, 32'});0.4480;089.53
8;Artificial neural networks;univariate;14;OrderedDict({'activation': 'relu', 'alpha': 0.0001, 'batch_size': 573, 'hidden_layer_sizes': '32, 32'});0.3982;137.80
9;Artificial neural networks;univariate;16;OrderedDict({'activation': 'relu', 'alpha': 0.01, 'batch_size': 256, 'hidden_layer_sizes': '64, 32'});0.3772;148.99
10;Artificial neural networks;wrapper;8;OrderedDict({'activation': 'identity', 'alpha': 0.0033704456334576983, 'batch_size': 644, 'hidden_layer_sizes': '64, 64'});0.4514;066.76
11;Artificial neural networks;wrapper;10;OrderedDict({'activation': 'relu', 'alpha': 0.0006407130905485844, 'batch_size': 562, 'hidden_layer_sizes': '64, 64'});0.4619;094.61
12;Artificial neural networks;wrapper;12;OrderedDict({'activation': 'identity', 'alpha': 0.0022509069396752495, 'batch_size': 1503, 'hidden_layer_sizes': '64, 64'});0.4290;053.14
13;Artificial neural networks;wrapper;14;OrderedDict({'activation': 'tanh', 'alpha': 0.0021227646072116114, 'batch_size': 462, 'hidden_layer_sizes': '32, 64, 32'});0.4584;186.02
14;Artificial neural networks;wrapper;16;OrderedDict({'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 415, 'hidden_layer_sizes': '64, 64'});0.4373;175.08
15;Support vector machines;multivariate;8;OrderedDict({'C': 96.18398405840975, 'gamma': 'scale', 'kernel': 'rbf'});0.4102;002.41
16;Support vector machines;multivariate;10;OrderedDict({'C': 82.90546877885846, 'gamma': 'scale', 'kernel': 'rbf'});0.4186;001.98
17;Support vector machines;multivariate;12;OrderedDict({'C': 73.65234245085956, 'gamma': 'scale', 'kernel': 'linear'});0.3048;164.02
18;Support vector machines;multivariate;14;OrderedDict({'C': 83.0457315447555, 'gamma': 'scale', 'kernel': 'rbf'});0.3956;052.42
19;Support vector machines;multivariate;16;OrderedDict({'C': 56.782865442341816, 'gamma': 'auto', 'kernel': 'rbf'});0.3916;001.54
20;Support vector machines;univariate;8;OrderedDict({'C': 60.43936696071759, 'gamma': 'scale', 'kernel': 'rbf'});0.4168;003.04
21;Support vector machines;univariate;10;OrderedDict({'C': 49.2794067550937, 'gamma': 'auto', 'kernel': 'rbf'});0.3908;022.68
22;Support vector machines;univariate;12;OrderedDict({'C': 50.63243829994258, 'gamma': 'scale', 'kernel': 'rbf'});0.3985;001.57
23;Support vector machines;univariate;14;OrderedDict({'C': 58.08196546177063, 'gamma': 'auto', 'kernel': 'linear'});0.3048;173.85
24;Support vector machines;univariate;16;OrderedDict({'C': 53.57606702721132, 'gamma': 'auto', 'kernel': 'rbf'});0.3732;079.02
25;Support vector machines;wrapper;8;OrderedDict({'C': 68.42622378245696, 'gamma': 'scale', 'kernel': 'rbf'});0.4108;002.45
26;Support vector machines;wrapper;10;OrderedDict({'C': 5.53453009201796, 'gamma': 'auto', 'kernel': 'rbf'});0.3139;001.59
27;Support vector machines;wrapper;12;OrderedDict({'C': 60.9607704374378, 'gamma': 'auto', 'kernel': 'rbf'});0.3546;002.12
28;Support vector machines;wrapper;14;OrderedDict({'C': 34.02024633416418, 'gamma': 'scale', 'kernel': 'rbf'});0.3933;007.43
29;Support vector machines;wrapper;16;OrderedDict({'C': 27.90304484687713, 'gamma': 'auto', 'kernel': 'rbf'});0.3950;003.95
5;Classification trees;wrapper;8;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 6, 'min_samples_leaf': 0.055047058414022104, 'min_samples_split': 0.057011229675801356, 'splitter': 'best'});0.4181;125.07
6;Classification trees;wrapper;10;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 5, 'min_samples_leaf': 0.023893974131417296, 'min_samples_split': 1e-06, 'splitter': 'random'});0.4159;133.97
7;Classification trees;wrapper;12;OrderedDict({'ccp_alpha': 0.002172825621076458, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 17, 'min_samples_leaf': 0.0015461155038294892, 'min_samples_split': 0.0963689557477803, 'splitter': 'random'});0.4231;119.40
8;Classification trees;wrapper;14;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 46, 'min_samples_leaf': 0.07216575873788625, 'min_samples_split': 0.1, 'splitter': 'random'});0.4182;138.22
9;Classification trees;wrapper;16;OrderedDict({'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 36, 'min_samples_leaf': 0.05161686299390667, 'min_samples_split': 0.04141045327164274, 'splitter': 'best'});0.4181;138.65
"""

stats = pd.read_csv(StringIO(output), delimiter=";")
print(stats.columns)
best_models = stats.loc[stats.groupby('model')['score'].idxmax()]
best_models

Index(['it', 'model', 'fss strategy', 'num fea', 'best parameters', 'score',
       'elapsed time '],
      dtype='object')


,it,model,fss strategy,num fea,best parameters,score,elapsed time
31,11,Artificial neural networks,wrapper,10,"OrderedDict({'activation': 'relu', 'alpha': 0....",0.4619,94.61
57,7,Classification trees,wrapper,12,OrderedDict({'ccp_alpha': 0.002172825621076458...,0.4231,119.40
53,3,K-nearest neighbors,wrapper,14,"OrderedDict({'metric': 'minkowski', 'n_neighbo...",0.8460,547.66
36,16,Support vector machines,multivariate,10,"OrderedDict({'C': 82.90546877885846, 'gamma': ...",0.4186,1.98


In [12]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import ast

print("model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall,iteration")
for index, row in best_models.iterrows():
    if row['model'] not in ['Support vector machines']:
        continue 
        
    model_name = row['model']
    k = int(row['num fea'])
    fss_strategy = row['fss strategy']
    kwargs = row['best parameters'].replace("OrderedDict(", '').replace("})", '}')
    kwargs = ast.literal_eval(kwargs)
    
    if model_name == 'Classification trees': 
        model = DecisionTreeClassifier(random_state=rng, **kwargs)
    elif model_name == 'K-nearest neighbors': 
        model = KNeighborsClassifier(**kwargs)
    elif model_name == 'Artificial neural networks': 
        model = BayesOptimizableMLPC(**kwargs, random_state=rng, max_iter=1000,warm_start=True, solver='adam')
    elif model_name == 'Support vector machines': 
        model = SVC(**kwargs, random_state=rng, cache_size=20000)
    else:
        raise ValueError("Unknown model")
    
    if model_name == 'Rule Induction':
        y_trans = y_train
        y_trans[y_trans == 2] = 1
        fea_subset = np.array(features)[fea_subsets[fss_strategy][k]]
        X = X_train[fea_subset]
        model.fit(X_train, y_trans)  # Fit the model on the transformed data
    else:
        if model_name == "Artificial neural networks":
            for it in range(10000):
                for k in range(20):
                    model.partial_fit(X_train, y_train, classes=[0, 1, 2]) 
                y_pred = model.predict(X_test)
                
                accuracy = accuracy_score(y_test, y_pred)
                matrix = confusion_matrix(y_test, y_pred)
                accuracy_per_class = matrix.diagonal()/matrix.sum(axis=1)
                f1_macro = f1_score(y_test, y_pred, average='macro')
                precision = precision_score(y_test, y_pred, average='macro')
                recall = recall_score(y_test, y_pred, average='macro')
                
                # Print the metrics
                print(f"{model_name},{accuracy:.03f},{",".join(f"{x:.3f}" for x in accuracy_per_class)},{f1_macro:.03f},{precision:.03f},{recall:.03f},{it}")
        elif model_name == "Support vector machines":
                # Generate a range of values that grows exponentially from 0 to just under 1
                num_points = 20  # Adjust as needed
                exponential_range = np.exp(np.linspace(0, -5, num_points))[::-1]
                
                # Normalize it to stay within 0 and 1
                exponential_range /= exponential_range.max()  
                exponential_range *= 0.2
                print(exponential_range)
                
                for it in exponential_range:
                    X_fraction, _, y_fraction, _ = train_test_split(X_train, y_train, test_size=1-it, stratify=y_train, random_state=rng)
                    print(f"Training with  {X_fraction.shape}", end="")
                    model.fit(X_fraction, y_fraction) 
                    print("\rTrained...", end="")
                    y_pred = model.predict(X_test)
                    
                    accuracy = accuracy_score(y_test, y_pred)
                    matrix = confusion_matrix(y_test, y_pred)
                    accuracy_per_class = matrix.diagonal()/matrix.sum(axis=1)
                    f1_macro = f1_score(y_test, y_pred, average='macro')
                    precision = precision_score(y_test, y_pred, average='macro')
                    recall = recall_score(y_test, y_pred, average='macro')
                    
                    # Print the metrics
                    print(f"\r{model_name},{accuracy:.03f},{",".join(f"{x:.3f}" for x in accuracy_per_class)},{f1_macro:.03f},{precision:.03f},{recall:.03f},{it}")
        else:
            model.fit(X_train, y_train) 
        
    y_pred = model.predict(X_test)
    
    # Calculate the metrics
    if model_name == 'Rule Induction':
        y_trans = y_test
        y_trans[y_trans == 2] = 1
        accuracy = accuracy_score(y_test, y_pred)
        matrix = confusion_matrix(y_test, y_pred)
    else:
        accuracy = accuracy_score(y_test, y_pred)
        matrix = confusion_matrix(y_test, y_pred)
        
    accuracy_per_class = matrix.diagonal()/matrix.sum(axis=1)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    
    # Print the metrics
    if model_name == 'Rule Induction':
        print(f"{model_name},{accuracy:.03f},{",".join(f"{x:.3f}" for x in accuracy_per_class)},Null,{f1_macro:.03f},{precision:.03f},{recall:.03f},Null")
    else:
        print(f"{model_name},{accuracy:.03f},{",".join(f"{x:.3f}" for x in accuracy_per_class)},{f1_macro:.03f},{precision:.03f},{recall:.03f},Null")
    

model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall,iteration
[0.00133759 0.00174326 0.00227104 0.00295771 0.00385109 0.0050134
 0.0065256  0.00849303 0.01105272 0.01438295 0.01871569 0.02435272
 0.03168669 0.04122841 0.05364249 0.06979361 0.09080674 0.1181455
 0.15371411 0.19999   ]
Support vector machines,0.793,0.897,0.009,0.267,0.394,0.399,0.391,0.0013375893998170934
Support vector machines,0.790,0.893,0.009,0.267,0.392,0.396,0.390,0.0017432571056736554
Support vector machines,0.786,0.892,0.006,0.249,0.384,0.387,0.382,0.0022710438246341044
Support vector machines,0.798,0.899,0.017,0.288,0.406,0.412,0.402,0.0029577113031874313
Support vector machines,0.779,0.877,0.041,0.284,0.402,0.403,0.401,0.0038510877546288234
Support vector machines,0.793,0.896,0.015,0.274,0.398,0.403,0.395,0.0050133992211213105
Support vector machines,0.792,0.890,0.015,0.303,0.404,0.407,0.403,0.006525603782770808
Support vector machines,0.796,0.899,0.022,0.275,0.403,0.410,0.399,0.0084930305

In [15]:
output = """model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall,iteration
Classification trees,0.570,0.566,0.287,0.636,0.392,0.431,0.496,Null
K-nearest neighbors,0.843,0.979,0.000,0.132,0.374,0.450,0.370,Null
Support vector machines,0.793,0.897,0.009,0.267,0.394,0.399,0.391,0.0013375893998170934
Support vector machines,0.790,0.893,0.009,0.267,0.392,0.396,0.390,0.0017432571056736554
Support vector machines,0.786,0.892,0.006,0.249,0.384,0.387,0.382,0.0022710438246341044
Support vector machines,0.798,0.899,0.017,0.288,0.406,0.412,0.402,0.0029577113031874313
Support vector machines,0.779,0.877,0.041,0.284,0.402,0.403,0.401,0.0038510877546288234
Support vector machines,0.793,0.896,0.015,0.274,0.398,0.403,0.395,0.0050133992211213105
Support vector machines,0.792,0.890,0.015,0.303,0.404,0.407,0.403,0.006525603782770808
Support vector machines,0.796,0.899,0.022,0.275,0.403,0.410,0.399,0.008493030502885964
Support vector machines,0.802,0.905,0.004,0.280,0.399,0.405,0.397,0.011052715876934098
Support vector machines,0.806,0.913,0.013,0.261,0.401,0.412,0.396,0.0143829487883446
Support vector machines,0.809,0.914,0.011,0.279,0.407,0.418,0.401,0.01871568880267776
Support vector machines,0.808,0.917,0.009,0.257,0.400,0.412,0.394,0.024352722767323982
Support vector machines,0.817,0.929,0.015,0.242,0.404,0.425,0.395,0.03168668506680567
Support vector machines,0.819,0.932,0.017,0.242,0.406,0.430,0.397,0.04122840565028178
Support vector machines,0.826,0.944,0.022,0.223,0.408,0.443,0.396,0.053642490693992205
Support vector machines,0.825,0.941,0.019,0.231,0.408,0.440,0.397,0.06979361418626401
Support vector machines,0.828,0.948,0.011,0.211,0.400,0.436,0.390,0.09080674476690055
Support vector machines,0.831,0.954,0.011,0.197,0.397,0.442,0.387,0.11814550278024634
Support vector machines,0.835,0.959,0.009,0.198,0.399,0.453,0.389,0.15371410531874716
Support vector machines,0.835,0.959,0.017,0.187,0.400,0.463,0.388,0.19999
Artificial neural networks,0.849,0.971,0.000,0.221,0.410,0.468,0.397,0
Artificial neural networks,0.847,0.978,0.000,0.171,0.392,0.468,0.383,1
Artificial neural networks,0.847,0.965,0.000,0.243,0.415,0.461,0.403,2
Artificial neural networks,0.849,0.978,0.000,0.178,0.395,0.471,0.385,3
Artificial neural networks,0.847,0.974,0.000,0.188,0.397,0.462,0.387,4
Artificial neural networks,0.849,0.976,0.000,0.187,0.399,0.471,0.388,5
Artificial neural networks,0.846,0.975,0.000,0.179,0.394,0.460,0.384,6
Artificial neural networks,0.847,0.968,0.000,0.224,0.409,0.461,0.397,7
Artificial neural networks,0.846,0.974,0.000,0.179,0.393,0.459,0.384,8
Artificial neural networks,0.846,0.972,0.000,0.190,0.397,0.459,0.387,9
Artificial neural networks,0.846,0.968,0.000,0.221,0.408,0.461,0.396,10
Artificial neural networks,0.846,0.965,0.000,0.239,0.413,0.459,0.401,11
Artificial neural networks,0.847,0.971,0.000,0.208,0.404,0.463,0.393,12
Artificial neural networks,0.847,0.978,0.000,0.164,0.389,0.466,0.381,13
Artificial neural networks,0.845,0.960,0.000,0.263,0.420,0.458,0.407,14
Artificial neural networks,0.846,0.976,0.000,0.175,0.393,0.462,0.384,15
Artificial neural networks,0.846,0.974,0.000,0.184,0.396,0.461,0.386,16
Artificial neural networks,0.847,0.978,0.000,0.166,0.389,0.463,0.381,17
Artificial neural networks,0.847,0.973,0.000,0.190,0.398,0.463,0.388,18
Artificial neural networks,0.847,0.971,0.000,0.212,0.406,0.463,0.394,19
Artificial neural networks,0.845,0.958,0.002,0.266,0.421,0.539,0.409,20
Artificial neural networks,0.845,0.963,0.000,0.241,0.413,0.457,0.401,21
Artificial neural networks,0.847,0.974,0.002,0.188,0.399,0.630,0.388,22
Artificial neural networks,0.846,0.973,0.004,0.190,0.401,0.572,0.389,23
Artificial neural networks,0.846,0.969,0.002,0.210,0.406,0.570,0.394,24
Artificial neural networks,0.845,0.966,0.004,0.222,0.410,0.589,0.398,25
Artificial neural networks,0.846,0.968,0.002,0.215,0.407,0.526,0.395,26
Artificial neural networks,0.844,0.961,0.002,0.243,0.414,0.537,0.402,27
Artificial neural networks,0.845,0.973,0.002,0.183,0.396,0.623,0.386,28
Artificial neural networks,0.847,0.975,0.002,0.182,0.397,0.631,0.386,29
Artificial neural networks,0.843,0.958,0.000,0.262,0.418,0.453,0.407,30
Artificial neural networks,0.846,0.967,0.004,0.227,0.413,0.683,0.399,31
Artificial neural networks,0.844,0.963,0.002,0.233,0.411,0.620,0.399,32
Artificial neural networks,0.844,0.963,0.002,0.238,0.413,0.502,0.401,33
Artificial neural networks,0.846,0.977,0.002,0.164,0.390,0.628,0.381,34
Artificial neural networks,0.845,0.976,0.000,0.164,0.388,0.457,0.380,35
Artificial neural networks,0.844,0.970,0.002,0.193,0.399,0.565,0.388,36
Artificial neural networks,0.844,0.962,0.000,0.241,0.412,0.453,0.401,37
Artificial neural networks,0.843,0.963,0.002,0.230,0.410,0.563,0.399,38
Artificial neural networks,0.845,0.973,0.002,0.184,0.397,0.495,0.387,39
Artificial neural networks,0.846,0.966,0.000,0.231,0.411,0.458,0.399,40
Artificial neural networks,0.845,0.976,0.002,0.164,0.389,0.506,0.381,41
Artificial neural networks,0.843,0.965,0.002,0.218,0.407,0.535,0.395,42
Artificial neural networks,0.844,0.960,0.002,0.248,0.415,0.564,0.403,43
Artificial neural networks,0.842,0.956,0.002,0.265,0.419,0.534,0.408,44
Artificial neural networks,0.845,0.962,0.002,0.246,0.416,0.566,0.403,45
Artificial neural networks,0.845,0.969,0.002,0.203,0.403,0.567,0.391,46
Artificial neural networks,0.844,0.966,0.002,0.218,0.407,0.501,0.395,47
Artificial neural networks,0.843,0.962,0.000,0.240,0.412,0.453,0.400,48
Artificial neural networks,0.843,0.966,0.002,0.210,0.403,0.498,0.393,49
Artificial neural networks,0.845,0.969,0.002,0.209,0.405,0.523,0.393,50
Artificial neural networks,0.844,0.964,0.002,0.226,0.409,0.521,0.398,51
Artificial neural networks,0.844,0.971,0.002,0.190,0.398,0.520,0.388,52
Artificial neural networks,0.844,0.964,0.002,0.225,0.409,0.490,0.397,53
Artificial neural networks,0.844,0.969,0.002,0.201,0.402,0.487,0.391,54
Artificial neural networks,0.843,0.970,0.002,0.185,0.395,0.506,0.386,55
Artificial neural networks,0.844,0.964,0.004,0.233,0.413,0.565,0.400,56
Artificial neural networks,0.845,0.965,0.002,0.229,0.411,0.487,0.399,57
Artificial neural networks,0.846,0.974,0.002,0.181,0.396,0.571,0.386,58
Artificial neural networks,0.845,0.969,0.002,0.201,0.402,0.522,0.391,59
Artificial neural networks,0.843,0.966,0.004,0.206,0.403,0.492,0.392,60
Artificial neural networks,0.845,0.973,0.002,0.182,0.396,0.499,0.386,61
Artificial neural networks,0.843,0.962,0.002,0.233,0.410,0.492,0.399,62
Artificial neural networks,0.845,0.969,0.002,0.207,0.404,0.505,0.393,63
Artificial neural networks,0.845,0.968,0.004,0.210,0.406,0.538,0.394,64
Artificial neural networks,0.843,0.968,0.000,0.199,0.399,0.451,0.389,65
Artificial neural networks,0.846,0.974,0.002,0.181,0.396,0.501,0.386,66
Artificial neural networks,0.843,0.958,0.002,0.255,0.417,0.500,0.405,67
Artificial neural networks,0.845,0.973,0.004,0.184,0.398,0.532,0.387,68
Artificial neural networks,0.844,0.964,0.004,0.226,0.410,0.536,0.398,69
Artificial neural networks,0.844,0.965,0.004,0.223,0.410,0.587,0.397,70
Artificial neural networks,0.844,0.967,0.002,0.211,0.405,0.495,0.393,71
Artificial neural networks,0.843,0.969,0.004,0.192,0.399,0.496,0.388,72
Artificial neural networks,0.844,0.972,0.002,0.181,0.395,0.484,0.385,73
Artificial neural networks,0.843,0.966,0.000,0.208,0.401,0.450,0.391,74
Artificial neural networks,0.842,0.959,0.000,0.247,0.413,0.450,0.402,75
Artificial neural networks,0.844,0.967,0.004,0.213,0.407,0.502,0.395,76
Artificial neural networks,0.842,0.960,0.002,0.243,0.413,0.479,0.402,77
Artificial neural networks,0.844,0.966,0.002,0.214,0.405,0.495,0.394,78
Artificial neural networks,0.844,0.972,0.000,0.184,0.394,0.454,0.385,79
Artificial neural networks,0.842,0.961,0.002,0.237,0.412,0.480,0.400,80
Artificial neural networks,0.844,0.968,0.004,0.200,0.402,0.547,0.391,81
Artificial neural networks,0.844,0.967,0.004,0.213,0.407,0.506,0.395,82
Artificial neural networks,0.844,0.971,0.002,0.182,0.395,0.499,0.385,83
Artificial neural networks,0.843,0.960,0.002,0.243,0.413,0.492,0.402,84
Artificial neural networks,0.846,0.966,0.004,0.228,0.413,0.542,0.400,85
Artificial neural networks,0.842,0.960,0.004,0.240,0.414,0.501,0.401,86
Artificial neural networks,0.844,0.963,0.004,0.230,0.412,0.548,0.399,87
Artificial neural networks,0.846,0.974,0.004,0.182,0.397,0.525,0.387,88
Artificial neural networks,0.844,0.969,0.004,0.196,0.401,0.508,0.390,89
Artificial neural networks,0.844,0.969,0.004,0.196,0.401,0.520,0.390,90
Artificial neural networks,0.843,0.969,0.004,0.191,0.399,0.533,0.388,91
Artificial neural networks,0.844,0.970,0.002,0.197,0.400,0.520,0.389,92
Artificial neural networks,0.843,0.965,0.000,0.211,0.402,0.450,0.392,93
Artificial neural networks,0.844,0.970,0.002,0.195,0.399,0.537,0.389,94
Artificial neural networks,0.840,0.951,0.004,0.280,0.423,0.492,0.412,95
Artificial neural networks,0.844,0.971,0.004,0.186,0.397,0.487,0.387,96
Artificial neural networks,0.844,0.967,0.002,0.214,0.406,0.509,0.394,97
Artificial neural networks,0.842,0.959,0.002,0.246,0.414,0.483,0.402,98
Artificial neural networks,0.843,0.966,0.000,0.211,0.403,0.451,0.392,99
Artificial neural networks,0.842,0.961,0.000,0.231,0.408,0.448,0.397,100
Artificial neural networks,0.844,0.968,0.002,0.204,0.402,0.500,0.391,101
Artificial neural networks,0.844,0.974,0.000,0.172,0.390,0.454,0.382,102
Artificial neural networks,0.842,0.964,0.000,0.215,0.403,0.449,0.393,103
Artificial neural networks,0.841,0.954,0.000,0.271,0.419,0.448,0.408,104
Artificial neural networks,0.844,0.967,0.000,0.211,0.403,0.454,0.393,105
Artificial neural networks,0.843,0.969,0.002,0.191,0.397,0.475,0.387,106
Artificial neural networks,0.845,0.970,0.002,0.197,0.400,0.489,0.390,107
Artificial neural networks,0.841,0.959,0.000,0.234,0.408,0.446,0.398,108
Artificial neural networks,0.844,0.970,0.002,0.197,0.400,0.480,0.390,109
Artificial neural networks,0.843,0.963,0.002,0.223,0.408,0.481,0.396,110
Artificial neural networks,0.842,0.961,0.002,0.233,0.410,0.496,0.399,111
Artificial neural networks,0.843,0.964,0.000,0.227,0.408,0.452,0.397,112
Artificial neural networks,0.842,0.963,0.000,0.222,0.405,0.448,0.395,113
Artificial neural networks,0.842,0.960,0.002,0.239,0.412,0.491,0.400,114
Artificial neural networks,0.841,0.959,0.000,0.237,0.409,0.447,0.399,115
Artificial neural networks,0.843,0.973,0.004,0.168,0.390,0.516,0.382,116
Artificial neural networks,0.844,0.970,0.000,0.193,0.397,0.452,0.388,117
Artificial neural networks,0.841,0.958,0.002,0.243,0.412,0.489,0.401,118
Artificial neural networks,0.844,0.969,0.000,0.198,0.399,0.453,0.389,119
Artificial neural networks,0.843,0.962,0.004,0.231,0.412,0.512,0.399,120
Artificial neural networks,0.842,0.959,0.004,0.244,0.415,0.532,0.402,121
Artificial neural networks,0.842,0.965,0.000,0.213,0.403,0.450,0.393,122
Artificial neural networks,0.841,0.958,0.002,0.246,0.414,0.476,0.402,123
Artificial neural networks,0.842,0.959,0.002,0.244,0.413,0.487,0.402,124
Artificial neural networks,0.845,0.976,0.004,0.159,0.388,0.530,0.380,125
Artificial neural networks,0.843,0.964,0.000,0.225,0.407,0.452,0.396,126
Artificial neural networks,0.845,0.977,0.000,0.159,0.386,0.457,0.378,127
Artificial neural networks,0.843,0.962,0.000,0.234,0.410,0.451,0.399,128
Artificial neural networks,0.844,0.971,0.002,0.186,0.396,0.494,0.386,129
Artificial neural networks,0.844,0.966,0.004,0.214,0.407,0.512,0.395,130
Artificial neural networks,0.842,0.961,0.000,0.233,0.409,0.449,0.398,131
Artificial neural networks,0.843,0.966,0.000,0.212,0.403,0.451,0.393,132
Artificial neural networks,0.841,0.958,0.002,0.241,0.412,0.468,0.401,133
Artificial neural networks,0.844,0.970,0.006,0.195,0.402,0.522,0.390,134
Artificial neural networks,0.841,0.957,0.000,0.252,0.413,0.447,0.403,135
Artificial neural networks,0.844,0.971,0.000,0.189,0.396,0.454,0.387,136
Artificial neural networks,0.842,0.962,0.002,0.231,0.410,0.471,0.398,137
Artificial neural networks,0.843,0.968,0.000,0.198,0.398,0.451,0.389,138
Artificial neural networks,0.844,0.966,0.002,0.217,0.407,0.496,0.395,139
Artificial neural networks,0.843,0.967,0.002,0.202,0.401,0.471,0.390,140
Artificial neural networks,0.842,0.961,0.000,0.233,0.409,0.449,0.398,141
Artificial neural networks,0.842,0.964,0.004,0.214,0.406,0.491,0.394,142
Artificial neural networks,0.844,0.967,0.004,0.210,0.405,0.508,0.394,143
Artificial neural networks,0.842,0.958,0.000,0.246,0.412,0.448,0.401,144
Artificial neural networks,0.842,0.961,0.002,0.233,0.410,0.472,0.399,145
Artificial neural networks,0.841,0.959,0.002,0.239,0.411,0.480,0.400,146
Artificial neural networks,0.841,0.962,0.004,0.218,0.406,0.491,0.395,147
Artificial neural networks,0.841,0.960,0.004,0.236,0.412,0.479,0.400,148
Artificial neural networks,0.841,0.958,0.004,0.246,0.415,0.503,0.403,149
Artificial neural networks,0.843,0.965,0.000,0.216,0.404,0.450,0.394,150
Artificial neural networks,0.844,0.971,0.000,0.183,0.394,0.453,0.385,151
Artificial neural networks,0.843,0.966,0.002,0.209,0.403,0.486,0.392,152
Artificial neural networks,0.844,0.969,0.002,0.193,0.398,0.476,0.388,153
Artificial neural networks,0.843,0.967,0.002,0.204,0.402,0.477,0.391,154
Artificial neural networks,0.845,0.976,0.002,0.159,0.387,0.490,0.379,155
Artificial neural networks,0.844,0.969,0.002,0.203,0.402,0.485,0.391,156
Artificial neural networks,0.843,0.961,0.000,0.235,0.410,0.450,0.399,157
Artificial neural networks,0.838,0.947,0.004,0.288,0.424,0.481,0.413,158
Artificial neural networks,0.844,0.966,0.002,0.215,0.406,0.480,0.395,159
Artificial neural networks,0.844,0.971,0.002,0.186,0.396,0.486,0.386,160
Artificial neural networks,0.843,0.968,0.002,0.199,0.400,0.486,0.390,161
Artificial neural networks,0.842,0.964,0.000,0.213,0.402,0.447,0.392,162
Artificial neural networks,0.844,0.969,0.000,0.196,0.398,0.452,0.388,163
Artificial neural networks,0.843,0.962,0.002,0.232,0.410,0.497,0.399,164
Artificial neural networks,0.844,0.969,0.002,0.196,0.399,0.478,0.389,165
Artificial neural networks,0.843,0.961,0.004,0.237,0.413,0.535,0.401,166
Artificial neural networks,0.841,0.962,0.000,0.224,0.405,0.447,0.395,167
Artificial neural networks,0.842,0.959,0.000,0.247,0.413,0.450,0.402,168
Artificial neural networks,0.844,0.969,0.002,0.201,0.401,0.475,0.390,169
Artificial neural networks,0.843,0.968,0.000,0.201,0.399,0.451,0.389,170
Artificial neural networks,0.844,0.970,0.002,0.193,0.399,0.495,0.388,171
Artificial neural networks,0.843,0.965,0.002,0.213,0.405,0.484,0.393,172
Artificial neural networks,0.842,0.962,0.002,0.229,0.409,0.560,0.398,173
Artificial neural networks,0.843,0.961,0.004,0.237,0.413,0.511,0.401,174
Artificial neural networks,0.844,0.970,0.000,0.192,0.397,0.453,0.388,175
Artificial neural networks,0.842,0.958,0.002,0.254,0.416,0.481,0.405,176
Artificial neural networks,0.842,0.961,0.004,0.231,0.411,0.499,0.399,177
Artificial neural networks,0.845,0.970,0.004,0.198,0.402,0.508,0.391,178
Artificial neural networks,0.843,0.963,0.004,0.224,0.409,0.488,0.397,179
Artificial neural networks,0.841,0.957,0.002,0.251,0.414,0.478,0.403,180
Artificial neural networks,0.843,0.964,0.002,0.225,0.408,0.485,0.397,181
Artificial neural networks,0.844,0.967,0.004,0.208,0.405,0.527,0.393,182
Artificial neural networks,0.843,0.965,0.002,0.219,0.407,0.489,0.395,183
Artificial neural networks,0.844,0.968,0.004,0.205,0.404,0.527,0.392,184
Artificial neural networks,0.844,0.969,0.004,0.202,0.403,0.505,0.392,185
Artificial neural networks,0.844,0.970,0.002,0.194,0.399,0.510,0.389,186
Artificial neural networks,0.842,0.963,0.002,0.224,0.407,0.515,0.396,187
Artificial neural networks,0.842,0.955,0.002,0.264,0.418,0.491,0.407,188
Artificial neural networks,0.844,0.969,0.006,0.194,0.402,0.523,0.390,189
Artificial neural networks,0.842,0.961,0.004,0.229,0.410,0.495,0.398,190
Artificial neural networks,0.844,0.966,0.002,0.217,0.407,0.483,0.395,191
Artificial neural networks,0.842,0.960,0.002,0.236,0.411,0.490,0.399,192
Artificial neural networks,0.842,0.962,0.004,0.226,0.409,0.514,0.397,193
Artificial neural networks,0.845,0.967,0.004,0.218,0.409,0.495,0.396,194
Artificial neural networks,0.842,0.970,0.002,0.184,0.394,0.490,0.385,195
Artificial neural networks,0.845,0.972,0.002,0.186,0.397,0.489,0.387,196
Artificial neural networks,0.844,0.971,0.002,0.189,0.398,0.477,0.387,197
Artificial neural networks,0.844,0.967,0.004,0.216,0.408,0.515,0.395,198
Artificial neural networks,0.838,0.951,0.004,0.265,0.418,0.538,0.407,199
Artificial neural networks,0.843,0.962,0.002,0.235,0.411,0.479,0.400,200
Artificial neural networks,0.843,0.964,0.002,0.221,0.407,0.483,0.396,201
Artificial neural networks,0.844,0.968,0.000,0.203,0.401,0.452,0.390,202
Artificial neural networks,0.843,0.967,0.000,0.203,0.400,0.450,0.390,203
Artificial neural networks,0.844,0.967,0.004,0.215,0.407,0.520,0.395,204
Artificial neural networks,0.844,0.967,0.000,0.208,0.402,0.452,0.392,205
Artificial neural networks,0.843,0.962,0.002,0.233,0.411,0.506,0.399,206
Artificial neural networks,0.841,0.956,0.000,0.259,0.416,0.448,0.405,207
Artificial neural networks,0.844,0.963,0.002,0.231,0.410,0.519,0.399,208
Artificial neural networks,0.842,0.964,0.000,0.218,0.404,0.449,0.394,209
Artificial neural networks,0.845,0.967,0.002,0.218,0.408,0.487,0.396,210
Artificial neural networks,0.840,0.956,0.004,0.246,0.414,0.500,0.402,211
Artificial neural networks,0.844,0.970,0.002,0.194,0.399,0.491,0.389,212
Artificial neural networks,0.843,0.963,0.004,0.225,0.410,0.496,0.398,213
Artificial neural networks,0.843,0.967,0.004,0.205,0.403,0.502,0.392,214
Artificial neural networks,0.844,0.968,0.000,0.207,0.402,0.454,0.392,215
Artificial neural networks,0.843,0.964,0.004,0.223,0.409,0.535,0.397,216
Artificial neural networks,0.844,0.967,0.002,0.209,0.404,0.500,0.393,217
Artificial neural networks,0.844,0.971,0.004,0.188,0.398,0.513,0.388,218
Artificial neural networks,0.842,0.960,0.004,0.239,0.413,0.498,0.401,219
Artificial neural networks,0.844,0.964,0.004,0.222,0.409,0.536,0.397,220
Artificial neural networks,0.843,0.965,0.004,0.213,0.406,0.486,0.394,221
Artificial neural networks,0.843,0.967,0.004,0.204,0.403,0.525,0.392,222
Artificial neural networks,0.841,0.956,0.004,0.259,0.418,0.508,0.406,223
Artificial neural networks,0.845,0.975,0.004,0.165,0.391,0.511,0.382,224
Artificial neural networks,0.841,0.957,0.004,0.252,0.416,0.509,0.404,225
Artificial neural networks,0.840,0.954,0.004,0.260,0.418,0.497,0.406,226
Artificial neural networks,0.845,0.977,0.004,0.162,0.390,0.553,0.381,227
Artificial neural networks,0.842,0.957,0.004,0.255,0.417,0.509,0.405,228
Artificial neural networks,0.843,0.963,0.004,0.229,0.411,0.486,0.399,229
Artificial neural networks,0.845,0.974,0.002,0.178,0.394,0.491,0.385,230
Artificial neural networks,0.843,0.961,0.004,0.239,0.414,0.503,0.401,231
Artificial neural networks,0.842,0.961,0.004,0.236,0.412,0.523,0.400,232
Artificial neural networks,0.845,0.969,0.004,0.206,0.405,0.538,0.393,233
Artificial neural networks,0.843,0.966,0.004,0.213,0.406,0.512,0.394,234
Artificial neural networks,0.841,0.959,0.004,0.239,0.412,0.507,0.401,235
Artificial neural networks,0.843,0.959,0.004,0.251,0.417,0.517,0.405,236
Artificial neural networks,0.845,0.969,0.004,0.202,0.404,0.516,0.392,237
Artificial neural networks,0.842,0.960,0.004,0.238,0.412,0.499,0.401,238
Artificial neural networks,0.841,0.956,0.002,0.259,0.417,0.469,0.406,239
Artificial neural networks,0.846,0.976,0.004,0.168,0.392,0.520,0.383,240
Artificial neural networks,0.844,0.966,0.004,0.218,0.408,0.509,0.396,241
Artificial neural networks,0.843,0.960,0.004,0.246,0.416,0.502,0.403,242
Artificial neural networks,0.842,0.958,0.004,0.250,0.416,0.494,0.404,243
Artificial neural networks,0.844,0.966,0.004,0.215,0.407,0.520,0.395,244
Artificial neural networks,0.843,0.964,0.004,0.225,0.410,0.507,0.398,245
Artificial neural networks,0.842,0.958,0.004,0.256,0.418,0.498,0.406,246
Artificial neural networks,0.844,0.965,0.002,0.220,0.407,0.481,0.396,247
Artificial neural networks,0.838,0.947,0.004,0.286,0.423,0.499,0.413,248
Artificial neural networks,0.842,0.959,0.004,0.241,0.414,0.496,0.401,249
Artificial neural networks,0.844,0.964,0.000,0.231,0.410,0.454,0.398,250
Artificial neural networks,0.842,0.960,0.002,0.236,0.411,0.485,0.399,251
Artificial neural networks,0.843,0.963,0.004,0.229,0.411,0.489,0.399,252
Artificial neural networks,0.843,0.963,0.000,0.231,0.409,0.452,0.398,253
Artificial neural networks,0.839,0.952,0.000,0.266,0.416,0.444,0.406,254
Artificial neural networks,0.844,0.967,0.004,0.209,0.405,0.519,0.393,255
Artificial neural networks,0.843,0.966,0.004,0.211,0.406,0.503,0.394,256
Artificial neural networks,0.844,0.967,0.004,0.211,0.406,0.513,0.394,257
Artificial neural networks,0.840,0.955,0.004,0.256,0.417,0.485,0.405,258
Artificial neural networks,0.843,0.965,0.004,0.218,0.408,0.493,0.396,259
Artificial neural networks,0.844,0.968,0.004,0.201,0.402,0.513,0.391,260
Artificial neural networks,0.841,0.959,0.004,0.236,0.412,0.502,0.400,261
Artificial neural networks,0.842,0.956,0.002,0.257,0.417,0.482,0.405,262
Artificial neural networks,0.844,0.965,0.004,0.222,0.409,0.505,0.397,263
Artificial neural networks,0.846,0.973,0.004,0.183,0.398,0.503,0.387,264
Artificial neural networks,0.844,0.967,0.004,0.209,0.405,0.512,0.393,265
Artificial neural networks,0.844,0.963,0.004,0.234,0.413,0.520,0.400,266
Artificial neural networks,0.845,0.967,0.004,0.212,0.407,0.529,0.394,267
Artificial neural networks,0.841,0.953,0.000,0.277,0.420,0.449,0.410,268
Artificial neural networks,0.843,0.964,0.002,0.222,0.408,0.486,0.396,269
Artificial neural networks,0.845,0.971,0.004,0.197,0.402,0.523,0.391,270
Artificial neural networks,0.844,0.972,0.002,0.186,0.396,0.510,0.386,271
Artificial neural networks,0.843,0.969,0.004,0.193,0.399,0.498,0.389,272
Artificial neural networks,0.844,0.969,0.004,0.199,0.402,0.521,0.391,273
Artificial neural networks,0.845,0.968,0.004,0.208,0.405,0.529,0.393,274
Artificial neural networks,0.843,0.959,0.004,0.250,0.417,0.534,0.404,275
Artificial neural networks,0.845,0.967,0.004,0.215,0.408,0.538,0.395,276
Artificial neural networks,0.840,0.953,0.004,0.265,0.419,0.488,0.407,277
Artificial neural networks,0.843,0.962,0.002,0.233,0.411,0.499,0.399,278
Artificial neural networks,0.844,0.969,0.002,0.203,0.402,0.496,0.391,279
Artificial neural networks,0.842,0.953,0.004,0.277,0.423,0.545,0.412,280
Artificial neural networks,0.843,0.963,0.004,0.231,0.411,0.526,0.399,281
Artificial neural networks,0.842,0.960,0.004,0.235,0.412,0.503,0.400,282
Artificial neural networks,0.844,0.969,0.004,0.199,0.402,0.527,0.391,283
Artificial neural networks,0.845,0.974,0.004,0.180,0.396,0.509,0.386,284
Artificial neural networks,0.845,0.968,0.004,0.209,0.406,0.511,0.394,285
Artificial neural networks,0.844,0.970,0.004,0.192,0.400,0.547,0.389,286
Artificial neural networks,0.844,0.964,0.004,0.231,0.412,0.537,0.400,287
Artificial neural networks,0.843,0.963,0.000,0.229,0.408,0.452,0.397,288
Artificial neural networks,0.845,0.967,0.004,0.214,0.408,0.522,0.395,289
Artificial neural networks,0.845,0.971,0.002,0.195,0.400,0.493,0.389,290
Artificial neural networks,0.845,0.971,0.004,0.197,0.402,0.531,0.391,291
Artificial neural networks,0.842,0.958,0.000,0.249,0.413,0.450,0.403,292
Artificial neural networks,0.844,0.966,0.004,0.212,0.406,0.536,0.394,293
Artificial neural networks,0.842,0.958,0.004,0.249,0.416,0.500,0.404,294
Artificial neural networks,0.844,0.970,0.004,0.192,0.400,0.504,0.389,295
Artificial neural networks,0.844,0.970,0.004,0.192,0.400,0.502,0.389,296
Artificial neural networks,0.844,0.966,0.004,0.214,0.407,0.504,0.395,297
Artificial neural networks,0.845,0.971,0.004,0.187,0.398,0.506,0.388,298
Artificial neural networks,0.840,0.953,0.004,0.270,0.421,0.488,0.409,299
Artificial neural networks,0.845,0.969,0.004,0.205,0.405,0.500,0.393,300
Artificial neural networks,0.843,0.970,0.002,0.188,0.397,0.492,0.387,301
Artificial neural networks,0.840,0.956,0.004,0.253,0.416,0.502,0.404,302
Artificial neural networks,0.842,0.958,0.002,0.251,0.415,0.490,0.404,303
Artificial neural networks,0.845,0.969,0.004,0.208,0.406,0.530,0.394,304
Artificial neural networks,0.843,0.967,0.004,0.204,0.403,0.524,0.392,305
Artificial neural networks,0.844,0.963,0.004,0.233,0.413,0.527,0.400,306
Artificial neural networks,0.845,0.972,0.004,0.187,0.399,0.531,0.388,307
Artificial neural networks,0.843,0.964,0.004,0.223,0.409,0.546,0.397,308
Artificial neural networks,0.843,0.962,0.002,0.232,0.410,0.493,0.399,309
Artificial neural networks,0.844,0.968,0.002,0.203,0.402,0.490,0.391,310
Artificial neural networks,0.845,0.964,0.004,0.234,0.414,0.523,0.401,311
Artificial neural networks,0.843,0.965,0.004,0.211,0.405,0.501,0.393,312
Artificial neural networks,0.845,0.970,0.004,0.203,0.404,0.540,0.392,313
Artificial neural networks,0.845,0.972,0.004,0.186,0.398,0.500,0.387,314
Artificial neural networks,0.842,0.958,0.002,0.250,0.415,0.487,0.403,315
Artificial neural networks,0.838,0.952,0.004,0.259,0.416,0.508,0.405,316
Artificial neural networks,0.845,0.970,0.004,0.197,0.402,0.539,0.390,317
Artificial neural networks,0.843,0.964,0.004,0.223,0.409,0.496,0.397,318
Artificial neural networks,0.841,0.959,0.004,0.235,0.411,0.490,0.399,319
Artificial neural networks,0.844,0.961,0.004,0.243,0.415,0.519,0.403,320
Artificial neural networks,0.844,0.966,0.004,0.216,0.408,0.536,0.395,321
Artificial neural networks,0.841,0.959,0.004,0.241,0.413,0.499,0.401,322
Artificial neural networks,0.844,0.970,0.000,0.190,0.396,0.453,0.387,323
Artificial neural networks,0.844,0.969,0.004,0.198,0.402,0.518,0.390,324
Artificial neural networks,0.842,0.959,0.002,0.244,0.413,0.485,0.402,325
Artificial neural networks,0.844,0.966,0.004,0.214,0.407,0.503,0.395,326
Artificial neural networks,0.845,0.974,0.004,0.173,0.394,0.522,0.384,327
Artificial neural networks,0.842,0.957,0.004,0.255,0.417,0.522,0.405,328
Artificial neural networks,0.840,0.954,0.004,0.258,0.417,0.501,0.406,329
Artificial neural networks,0.844,0.963,0.004,0.230,0.412,0.536,0.399,330
Artificial neural networks,0.843,0.963,0.004,0.229,0.411,0.507,0.399,331
Artificial neural networks,0.843,0.961,0.004,0.241,0.414,0.511,0.402,332
Artificial neural networks,0.844,0.971,0.004,0.189,0.399,0.505,0.388,333
Artificial neural networks,0.842,0.959,0.002,0.245,0.414,0.492,0.402,334
Artificial neural networks,0.845,0.976,0.004,0.163,0.390,0.503,0.381,335
Artificial neural networks,0.844,0.973,0.004,0.175,0.394,0.505,0.384,336
Artificial neural networks,0.840,0.952,0.004,0.271,0.420,0.494,0.409,337
Artificial neural networks,0.843,0.968,0.004,0.199,0.401,0.533,0.390,338
Artificial neural networks,0.844,0.970,0.004,0.195,0.401,0.499,0.390,339
Artificial neural networks,0.844,0.969,0.004,0.195,0.401,0.494,0.390,340
Artificial neural networks,0.843,0.964,0.004,0.221,0.409,0.535,0.397,341
Artificial neural networks,0.841,0.959,0.004,0.238,0.412,0.530,0.400,342
Artificial neural networks,0.838,0.947,0.004,0.286,0.423,0.517,0.412,343
Artificial neural networks,0.844,0.972,0.002,0.178,0.393,0.519,0.384,344
Artificial neural networks,0.843,0.964,0.004,0.218,0.407,0.516,0.396,345
Artificial neural networks,0.845,0.971,0.006,0.194,0.402,0.538,0.390,346
Artificial neural networks,0.842,0.959,0.002,0.247,0.414,0.487,0.403,347
Artificial neural networks,0.842,0.959,0.004,0.241,0.414,0.491,0.402,348
Artificial neural networks,0.843,0.964,0.004,0.224,0.410,0.512,0.397,349
Artificial neural networks,0.841,0.957,0.006,0.248,0.416,0.514,0.404,350
Artificial neural networks,0.842,0.960,0.002,0.236,0.411,0.496,0.399,351
Artificial neural networks,0.842,0.962,0.006,0.227,0.411,0.509,0.399,352
Artificial neural networks,0.840,0.957,0.004,0.243,0.413,0.529,0.402,353
Artificial neural networks,0.842,0.961,0.004,0.229,0.410,0.492,0.398,354
Artificial neural networks,0.844,0.969,0.004,0.200,0.402,0.498,0.391,355
Artificial neural networks,0.843,0.960,0.004,0.244,0.416,0.526,0.403,356
Artificial neural networks,0.844,0.966,0.004,0.214,0.407,0.504,0.395,357
Artificial neural networks,0.843,0.971,0.004,0.184,0.396,0.511,0.386,358
Artificial neural networks,0.844,0.966,0.004,0.220,0.409,0.502,0.397,359
Artificial neural networks,0.840,0.955,0.004,0.253,0.416,0.505,0.404,360
Artificial neural networks,0.844,0.969,0.004,0.197,0.401,0.508,0.390,361
Artificial neural networks,0.844,0.969,0.004,0.200,0.403,0.499,0.391,362
Artificial neural networks,0.841,0.960,0.004,0.232,0.411,0.580,0.399,363
Artificial neural networks,0.844,0.972,0.004,0.183,0.397,0.495,0.386,364
Artificial neural networks,0.843,0.964,0.004,0.225,0.410,0.546,0.398,365"""

stats = pd.read_csv(StringIO(output), delimiter=",")
print(stats.columns)
best_models = stats.loc[stats.groupby('model')['f1_macro'].idxmax()]
best_models

Index(['model', 'accuracy', 'accuracy 0', 'accuracy 1', 'accuracy 2',
       'f1_macro', 'precision', 'recall', 'iteration'],
      dtype='object')


,model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall,iteration
180,Artificial neural networks,0.838,0.947,0.004,0.288,0.424,0.481,0.413,158
0,Classification trees,0.570,0.566,0.287,0.636,0.392,0.431,0.496,Null
1,K-nearest neighbors,0.843,0.979,0.000,0.132,0.374,0.450,0.370,Null
16,Support vector machines,0.826,0.944,0.022,0.223,0.408,0.443,0.396,0.053642490693992205
